<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/Evaluation/Evaluation_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mandarjoshi90/triviaqa.git

Cloning into 'triviaqa'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70 (from 1)
Receiving objects: 100% (70/70), 20.60 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
%cd triviaqa

/content/triviaqa


In [3]:
!pip install -r requirements.txt

In [19]:
import subprocess
import os
import json
from tqdm import tqdm
import importlib
import argparse

In [6]:
triviaqa_root = "/content/triviaqa"

In [7]:
env = os.environ.copy()
env["PYTHONPATH"] = triviaqa_root

In [8]:
%cd ..

/content


In [9]:
def evaluate_triviaqa(dataset_file, prediction_file):
  script_path = "/content/triviaqa/evaluation/triviaqa_evaluation.py"
  result = subprocess.run([
    "python", script_path,
    "--dataset_file", dataset_file,
    "--prediction_file", prediction_file],
    env=env,
    capture_output=True,
    text=True,
  )
  print(result.stdout)

In [10]:
def get_answers(questions, model):
    answers = {}

    for i in tqdm(range(len(questions)), desc="Retrieving Answers"):
        question = questions[i]
        answers[question[0]] = model.invoke(question[1])
    return answers

In [11]:
def load_questions(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())

    questions = []

    for i in tqdm(range(len(data["Data"])),desc="Loading Questions"):
        question = data["Data"][i]
        questions.append([question["QuestionId"], question["Question"]])
    return questions

In [30]:
def load_model(path, model_name):
  module = importlib.import_module(path)
  model = getattr(module, model_name)
  return model()

In [31]:
def get_args():
    parser = argparse.ArgumentParser(
        description='Evaluation Chain for TriviaQA {}')
    parser.add_argument('--dataset_file', help='Dataset file')
    parser.add_argument('--model_file', help='Model file')
    args = parser.parse_args()
    return args

In [34]:
if __name__ == "__main__":
    #args = get_args()
    evaluation_path = "/content/datasets/evaluation_data.json"
    predictions_path = "/content/datasets/predictions.json"
    model_path = "model.dummy_model"
    model_name = "dummy_model"
    questions = load_questions(evaluation_path)
    model = load_model(model_path, model_name)
    print(model.__class__)
    answers = get_answers(questions, model)
    with open(predictions_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(answers))#
    print("Test results:")
    evaluate_triviaqa(dataset_file, prediction_file)


Loading Questions: 100%|██████████| 7900/7900 [00:00<00:00, 498263.21it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/content/dummy_model/dummy_data.json'